In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
import time
import csv

In [2]:
# Example of target with class indices
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
print(input)
print(target)
output = loss(input, target)
output.backward()
# Example of target with class probabilities
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5).softmax(dim=1)
print(input)
print(target)
output = loss(input, target)
output.backward()

tensor([[ 0.1239, -1.0304,  0.4300, -0.9945, -0.3301],
        [-0.0159,  0.6191, -1.5094,  1.3357, -0.4140],
        [-0.2078,  0.9042,  1.0843, -0.5593,  1.0310]], requires_grad=True)
tensor([3, 4, 3])
tensor([[-0.6426, -0.5941,  2.3853,  0.5970,  0.5549],
        [ 0.0985, -2.2387,  1.1621,  0.5385, -0.3755],
        [ 0.4703, -1.1071, -0.0324,  0.1179,  1.9590]], requires_grad=True)
tensor([[0.2869, 0.0828, 0.4386, 0.0566, 0.1351],
        [0.3380, 0.0533, 0.3348, 0.0947, 0.1792],
        [0.1814, 0.0098, 0.1816, 0.5194, 0.1077]])


In [3]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [4]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [5]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [6]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [7]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [8]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([8])


In [9]:
loss_fn = nn.CrossEntropyLoss()
# loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [10]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        # print("pred: ",pred.shape)
        # print("y:",y.shape)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [11]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [12]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    epoch_start = time.perf_counter()
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.311877  [   64/60000]
loss: 2.289395  [ 6464/60000]
loss: 2.268669  [12864/60000]
loss: 2.260501  [19264/60000]
loss: 2.240828  [25664/60000]
loss: 2.233072  [32064/60000]
loss: 2.220690  [38464/60000]
loss: 2.195477  [44864/60000]
loss: 2.181258  [51264/60000]
loss: 2.157628  [57664/60000]
Test Error: 
 Accuracy: 52.5%, Avg loss: 2.147481 

Epoch 2
-------------------------------
loss: 2.155384  [   64/60000]
loss: 2.146036  [ 6464/60000]
loss: 2.081338  [12864/60000]
loss: 2.103990  [19264/60000]
loss: 2.049531  [25664/60000]
loss: 2.000629  [32064/60000]
loss: 2.012798  [38464/60000]
loss: 1.937223  [44864/60000]
loss: 1.930188  [51264/60000]
loss: 1.868556  [57664/60000]
Test Error: 
 Accuracy: 61.7%, Avg loss: 1.863560 

Epoch 3
-------------------------------
loss: 1.893412  [   64/60000]
loss: 1.864935  [ 6464/60000]
loss: 1.738046  [12864/60000]
loss: 1.788234  [19264/60000]
loss: 1.679261  [25664/60000]
loss: 1.634690  [32064/600

In [13]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [14]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>